In [1]:
pip install implicit

     |████████████████████████████████| 1.1 MB 1.0 MB/s eta 0:00:01
  Created wheel for implicit: filename=implicit-0.4.4-cp38-cp38-macosx_10_9_x86_64.whl size=761792 sha256=839f5987aba7b931ba6b7fb4cbf12d84140730f2ebf08fea995731d11e3c61a8
  Stored in directory: /Users/tabea/Library/Caches/pip/wheels/00/ac/67/6f4536c819ed560c2c7e17c0f7a920e3e50c26108616087d05
Successfully built implicit
Note: you may need to restart the kernel to use updated packages.


In [1]:
conda install -c conda-forge implicit

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [32]:
#imports 
import pandas as pd 
import numpy as np
import sys
import pandas as pd
import numpy as np
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
import random
import implicit
from sklearn.preprocessing import MinMaxScaler
from scipy import stats
from sklearn import metrics
from itertools import combinations 

# Load, Analyze and Preprocess Data

In [168]:
#load csv file
df_amazon_reviews = pd.read_csv('final_dataset.csv', dtype=str)
df_amazon_reviews.head()

,Unnamed: 0,overall,reviewerID,asin,reviewText,summary,category
0,0,4.0,A2C9XE9I8RSKNX,0887248845,Nice and snazzy to put on letters to a sponsor...,I'm pleased,Arts_Crafts_and_Sewing
1,1,5.0,A1WGU1NUVWJPR0,0988174979,This book has so many quilting possibilities! ...,LOVE IT! Blocks are fun and easy to make!,Arts_Crafts_and_Sewing
2,2,5.0,A333SV82Y549LK,0988174979,"What can I say, I LOVE this book of farm-relat...",Adorable and Easy to do Quilt Designs,Arts_Crafts_and_Sewing
3,3,5.0,A39EMFHHUTUX8M,0988174979,This is a great book and will keep a quilter q...,This is a great book and will keep a quilter q...,Arts_Crafts_and_Sewing
4,4,4.0,A27WH4VW6VS7GC,0988174979,Good,Four Stars,Arts_Crafts_and_Sewing


In [170]:
#rename reviewerID to reviewer
df_amazon_reviews = df_amazon_reviews.rename({'reviewerID': 'reviewer'}, axis=1) 

#create list with customers and count the products they have bought for each category = purchases
df_grouped = df_amazon_reviews.groupby(['category','reviewer']).size().reset_index(name="purchases")
print("Restructered Data Set\n", df_grouped.head())
print("Length of Data Set", len(df_grouped))
print("Max Value of Purchases", df_grouped['purchases'].max())

df_grouped.head()

Restructered Data Set
                  category              reviewer  purchases
0  Arts_Crafts_and_Sewing  A0116143FGG14B3OZ7UG          4
1  Arts_Crafts_and_Sewing  A0228801ER3IBSLNOT5W          3
2  Arts_Crafts_and_Sewing  A0272662PGC9D57A0G5C          7
3  Arts_Crafts_and_Sewing  A03033388K5LWDWFS9R9          6
4  Arts_Crafts_and_Sewing  A0323662LGDA74F87HNI          9
Length of Data Set 2251715
Max Value of Purchases 1478


,category,reviewer,purchases
0,Arts_Crafts_and_Sewing,A0116143FGG14B3OZ7UG,4
1,Arts_Crafts_and_Sewing,A0228801ER3IBSLNOT5W,3
2,Arts_Crafts_and_Sewing,A0272662PGC9D57A0G5C,7
3,Arts_Crafts_and_Sewing,A03033388K5LWDWFS9R9,6
4,Arts_Crafts_and_Sewing,A0323662LGDA74F87HNI,9


In [138]:
#Filter outliers
q_low = df_grouped["purchases"].quantile(0.001)
q_hi  = df_grouped["purchases"].quantile(0.999)

df_grouped = df_grouped[(df_grouped["purchases"] < q_hi) & (df_grouped["purchases"] > q_low)]
print("Restructered Data Set\n", df_grouped.head())
print("Length of Data Set", len(df_grouped))
print("Max Value of Purchases", df_grouped['purchases'].max())

Restructered Data Set
                  category              reviewer  purchases
0  Arts_Crafts_and_Sewing  A0116143FGG14B3OZ7UG          4
1  Arts_Crafts_and_Sewing  A0228801ER3IBSLNOT5W          3
2  Arts_Crafts_and_Sewing  A0272662PGC9D57A0G5C          7
3  Arts_Crafts_and_Sewing  A03033388K5LWDWFS9R9          6
4  Arts_Crafts_and_Sewing  A0323662LGDA74F87HNI          9
Length of Data Set 954463
Max Value of Purchases 29


In [171]:
#Get Sparsity
n_reviewer = df_grouped.reviewer.unique().shape[0]
n_categories = df_grouped.category.unique().shape[0]

print('Number of reviewer: {}'.format(n_reviewer))
print('Number of categories: {}'.format(n_categories))
print('Sparsity: {:4.3f}%'.format(float(df_grouped.shape[0]) / float(n_reviewer*n_categories) * 100))

Number of reviewer: 1453531
Number of categories: 19
Sparsity: 8.153%


In [139]:
#Get all the combinations of two cateogries
categegory_list = df_grouped.category.unique()
combination_list = list(combinations(categegory_list, 2))

In [140]:
#add a fake reviewer to the dataset
lookup_table = combination_list
print("Number of Combinations ", len(combination_list))
for idx, combi in enumerate(combination_list):
    df_grouped = df_grouped.append({'category': combi[0], 'reviewer': 'combination'+str(idx+1), 'purchases': 4}, ignore_index=True)
    df_grouped = df_grouped.append({'category': combi[1], 'reviewer': 'combination'+str(idx+1), 'purchases': 4}, ignore_index=True)
    lookup_table[idx] = lookup_table[idx] + ('combination'+str(idx+1), )
print("New Length of Data Set", len(df_grouped))

Number of Combinations  171
New Length of Data Set 954805


In [163]:
df_lookUp = pd.DataFrame(list(lookup_table), columns=['category1','category2','combination'])
print("Length of LookUpTable", len(df_lookUp))

Length of LookUpTable 171


In [172]:
# Create a numeric reviewer_id and category_id column
df_grouped['reviewer'] = df_grouped['reviewer'].astype("category")
df_grouped['category'] = df_grouped['category'].astype("category")
df_grouped['reviewer_id'] = df_grouped['reviewer'].cat.codes
df_grouped['category_id'] = df_grouped['category'].cat.codes

# Create ALS Model

In [173]:
# The implicit library expects data as a item-user matrix so we
# create two matricies, one for fitting the model (item-user) 
# and one for recommendations (user-item)
sparse_item_user = sparse.csr_matrix((df_grouped['purchases'].astype(float), (df_grouped['category_id'].astype(np.int32), df_grouped['reviewer_id'].astype(np.int32))))
sparse_user_item = sparse.csr_matrix((df_grouped['purchases'].astype(float), (df_grouped['reviewer_id'].astype(np.int32), df_grouped['category_id'].astype(np.int32))))

In [174]:
# Initialize the als model and fit it using the sparse item-user matrix
model = implicit.als.AlternatingLeastSquares(factors=32, regularization=0.1, iterations=20)

In [175]:
# Calculate the confidence by multiplying it by our alpha value.
alpha_val = 15
data_conf = (sparse_item_user * alpha_val).astype('double')

In [176]:
#Fit the model
model.fit(data_conf)

# Create Test Data

In [177]:
def make_train(ratings, pct_test = 0.2):
    '''
    This function will take in the original user-item matrix and "mask" a percentage of the original ratings where a
    user-item interaction has taken place for use as a test set. The test set will contain all of the original ratings, 
    while the training set replaces the specified percentage of them with a zero in the original ratings matrix. 
    
    parameters: 
    
    ratings - the original ratings matrix from which you want to generate a train/test set. Test is just a complete
    copy of the original set. This is in the form of a sparse csr_matrix. 
    
    pct_test - The percentage of user-item interactions where an interaction took place that you want to mask in the 
    training set for later comparison to the test set, which contains all of the original ratings. 
    
    returns:
    
    training_set - The altered version of the original data with a certain percentage of the user-item pairs 
    that originally had interaction set back to zero.
    
    test_set - A copy of the original ratings matrix, unaltered, so it can be used to see how the rank order 
    compares with the actual interactions.
    
    user_inds - From the randomly selected user-item indices, which user rows were altered in the training data.
    This will be necessary later when evaluating the performance via AUC.
    '''
    test_set = ratings.copy() # Make a copy of the original set to be the test set. 
    test_set[test_set != 0] = 1 # Store the test set as a binary preference matrix
    training_set = ratings.copy() # Make a copy of the original data we can alter as our training set. 
    nonzero_inds = training_set.nonzero() # Find the indices in the ratings data where an interaction exists
    nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1])) # Zip these pairs together of user,item index into list
    random.seed(0) # Set the random seed to zero for reproducibility
    num_samples = int(np.ceil(pct_test*len(nonzero_pairs))) # Round the number of samples needed to the nearest integer
    samples = random.sample(nonzero_pairs, num_samples) # Sample a random number of user-item pairs without replacement
    user_inds = [index[0] for index in samples] # Get the user row indices
    item_inds = [index[1] for index in samples] # Get the item column indices
    training_set[user_inds, item_inds] = 0 # Assign all of the randomly chosen user-item pairs to zero
    training_set.eliminate_zeros() # Get rid of zeros in sparse array storage after update to save space
    return training_set, test_set, list(set(user_inds)) # Output the unique list of user rows that were altered  

In [178]:
product_train, product_test, product_users_altered = make_train(sparse_user_item, pct_test = 0.2)


# Evaluating the Model

In [179]:
def auc_score(predictions, test):
    '''
    This simple function will output the area under the curve using sklearn's metrics. 
    
    parameters:
    
    - predictions: your prediction output
    
    - test: the actual target result you are comparing to
    
    returns:
    
    - AUC (area under the Receiver Operating Characterisic curve)
    '''
    fpr, tpr, thresholds = metrics.roc_curve(test, predictions)
    return metrics.auc(fpr, tpr)  

def calc_mean_auc(training_set, altered_users, predictions, test_set):
    '''
    This function will calculate the mean AUC by user for any user that had their user-item matrix altered. 
    
    parameters:
    
    training_set - The training set resulting from make_train, where a certain percentage of the original
    user/item interactions are reset to zero to hide them from the model 
    
    predictions - The matrix of your predicted ratings for each user/item pair as output from the implicit MF.
    These should be stored in a list, with user vectors as item zero and item vectors as item one. 
    
    altered_users - The indices of the users where at least one user/item pair was altered from make_train function
    
    test_set - The test set constucted earlier from make_train function
    
    
    
    returns:
    
    The mean AUC (area under the Receiver Operator Characteristic curve) of the test set only on user-item interactions
    there were originally zero to test ranking ability in addition to the most popular items as a benchmark.
    '''
    
    
    store_auc = [] # An empty list to store the AUC for each user that had an item removed from the training set
    popularity_auc = [] # To store popular AUC scores
    pop_items = np.array(test_set.sum(axis = 0)).reshape(-1) # Get sum of item iteractions to find most popular
    item_vecs = predictions[1]
    for user in altered_users: # Iterate through each user that had an item altered
        training_row = training_set[user,:].toarray().reshape(-1) # Get the training set row
        zero_inds = np.where(training_row == 0) # Find where the interaction had not yet occurred
        # Get the predicted values based on our user/item vectors
        user_vec = predictions[0][user,:]
        pred = user_vec.dot(item_vecs).toarray()[0,zero_inds].reshape(-1)
        # Get only the items that were originally zero
        # Select all ratings from the MF prediction for this user that originally had no iteraction
        actual = test_set[user,:].toarray()[0,zero_inds].reshape(-1) 
        # Select the binarized yes/no interaction pairs from the original full data
        # that align with the same pairs in training 
        pop = pop_items[zero_inds] # Get the item popularity for our chosen items
        store_auc.append(auc_score(pred, actual)) # Calculate AUC for the given user and store
        popularity_auc.append(auc_score(pop, actual)) # Calculate AUC using most popular and score
    # End users iteration
    
    return float('%.3f'%np.mean(store_auc)), float('%.3f'%np.mean(popularity_auc))  
   # Return the mean AUC rounded to three decimal places for both test and popularity benchmark

In [180]:
calc_mean_auc(product_train, product_users_altered, 
              [sparse.csr_matrix(model.user_factors), sparse.csr_matrix(model.item_factors.T)], product_test)

(1.0, 0.83)

# Get Recommendations

In [145]:
# Find the 3 most similar to a specific category
item_id = 13
n_similar = 3

# Get the user and item vectors from our trained model
user_vecs = model.user_factors
item_vecs = model.item_factors

# Calculate the vector norms
item_norms = np.sqrt((item_vecs * item_vecs).sum(axis=1))

# Calculate the similarity score, grab the top N items and
# create a list of item-score tuples of most similar artists
scores = item_vecs.dot(item_vecs[item_id]) / item_norms
top_idx = np.argpartition(scores, -n_similar)[-n_similar:]
similar = sorted(zip(top_idx, scores[top_idx] / item_norms[item_id]), key=lambda x: -x[1])

# Print the names of our most similar artists
for item in similar:
    idx, score = item
    print (df_grouped.category.loc[df_grouped.category_id == idx].iloc[0])

Industrial_and_Scientific
Luxury_Beauty
All_Beauty


In [36]:
#recommend by user id and return recommended category id
def recommendByID(user_id, sparse_user_item, user_vecs, item_vecs, num_items=3):
    
    user_interactions = sparse_user_item[user_id,:].toarray()
    print(len(user_interactions))

    user_interactions = user_interactions.reshape(-1) + 1
    user_interactions[user_interactions > 1] = 0

    rec_vector = user_vecs[user_id,:].dot(item_vecs.T).toarray()

    min_max = MinMaxScaler()
    rec_vector_scaled = min_max.fit_transform(rec_vector.reshape(-1,1))[:,0]
    recommend_vector = user_interactions * rec_vector_scaled
    print(recommend_vector)
    item_idx = np.argsort(recommend_vector)[::-1][:num_items]

    categories = []
    scores = []

    for idx in item_idx:
        categories.append(df_grouped.category.loc[df_grouped.category_id == idx].iloc[0])
        scores.append(recommend_vector[idx])

    recommendations = pd.DataFrame({'categories': categories, 'score': scores})

    return recommend_vector

In [165]:
for index, combination in df_lookUp.iterrows():
    print(combination)
    print("For the categories:", combination['category1'],combination['category2'])
    # Use the implicit recommender:
    recommended = model.recommend(getIdByUser(combination['combination']), sparse_user_item)
    print("This category is recommended: ", getCatById(recommended[0][0]))
    df_lookUp.loc[index,'recommended_category'] = getCatById(recommended[0][0])

category1      Arts_Crafts_and_Sewing
category2                  Automotive
combination              combination1
Name: 0, dtype: object
For the categories: Arts_Crafts_and_Sewing Automotive
This category is recommended:  Luxury_Beauty
category1      Arts_Crafts_and_Sewing
category2               CDs_and_Vinyl
combination              combination2
Name: 1, dtype: object
For the categories: Arts_Crafts_and_Sewing CDs_and_Vinyl
This category is recommended:  Automotive
category1           Arts_Crafts_and_Sewing
category2      Cell_Phones_and_Accessories
combination                   combination3
Name: 2, dtype: object
For the categories: Arts_Crafts_and_Sewing Cell_Phones_and_Accessories
This category is recommended:  Kindle_Store
category1          Arts_Crafts_and_Sewing
category2      Clothing_Shoes_and_Jewelry
combination                  combination4
Name: 3, dtype: object
For the categories: Arts_Crafts_and_Sewing Clothing_Shoes_and_Jewelry
This category is recommended:  Automotive


This category is recommended:  Grocery_and_Gourmet_Food
category1                    CDs_and_Vinyl
category2      Cell_Phones_and_Accessories
combination                  combination36
Name: 35, dtype: object
For the categories: CDs_and_Vinyl Cell_Phones_and_Accessories
This category is recommended:  Kindle_Store
category1                   CDs_and_Vinyl
category2      Clothing_Shoes_and_Jewelry
combination                 combination37
Name: 36, dtype: object
For the categories: CDs_and_Vinyl Clothing_Shoes_and_Jewelry
This category is recommended:  Kindle_Store
category1      CDs_and_Vinyl
category2        Electronics
combination    combination38
Name: 37, dtype: object
For the categories: CDs_and_Vinyl Electronics
This category is recommended:  Grocery_and_Gourmet_Food
category1                 CDs_and_Vinyl
category2      Grocery_and_Gourmet_Food
combination               combination39
Name: 38, dtype: object
For the categories: CDs_and_Vinyl Grocery_and_Gourmet_Food
This category 

This category is recommended:  Patio_Lawn_and_Garden
category1      Clothing_Shoes_and_Jewelry
category2      Tools_and_Home_Improvement
combination                 combination78
Name: 77, dtype: object
For the categories: Clothing_Shoes_and_Jewelry Tools_and_Home_Improvement
This category is recommended:  Luxury_Beauty
category1      Clothing_Shoes_and_Jewelry
category2                  Toys_and_Games
combination                 combination79
Name: 78, dtype: object
For the categories: Clothing_Shoes_and_Jewelry Toys_and_Games
This category is recommended:  Tools_and_Home_Improvement
category1      Clothing_Shoes_and_Jewelry
category2                     Video_Games
combination                 combination80
Name: 79, dtype: object
For the categories: Clothing_Shoes_and_Jewelry Video_Games
This category is recommended:  Kindle_Store
category1                   Electronics
category2      Grocery_and_Gourmet_Food
combination               combination81
Name: 80, dtype: object
For the cat

This category is recommended:  Automotive
category1      Home_and_Kitchen
category2           Video_Games
combination      combination116
Name: 115, dtype: object
For the categories: Home_and_Kitchen Video_Games
This category is recommended:  Automotive
category1      Industrial_and_Scientific
category2                   Kindle_Store
combination               combination117
Name: 116, dtype: object
For the categories: Industrial_and_Scientific Kindle_Store
This category is recommended:  Luxury_Beauty
category1      Industrial_and_Scientific
category2                  Luxury_Beauty
combination               combination118
Name: 117, dtype: object
For the categories: Industrial_and_Scientific Luxury_Beauty
This category is recommended:  Movies_and_TV
category1      Industrial_and_Scientific
category2                  Movies_and_TV
combination               combination119
Name: 118, dtype: object
For the categories: Industrial_and_Scientific Movies_and_TV
This category is recommended:  Lu

This category is recommended:  Automotive
category1      Musical_Instruments
category2      Sports_and_Outdoors
combination         combination153
Name: 152, dtype: object
For the categories: Musical_Instruments Sports_and_Outdoors
This category is recommended:  Home_and_Kitchen
category1             Musical_Instruments
category2      Tools_and_Home_Improvement
combination                combination154
Name: 153, dtype: object
For the categories: Musical_Instruments Tools_and_Home_Improvement
This category is recommended:  Toys_and_Games
category1      Musical_Instruments
category2           Toys_and_Games
combination         combination155
Name: 154, dtype: object
For the categories: Musical_Instruments Toys_and_Games
This category is recommended:  Automotive
category1      Musical_Instruments
category2              Video_Games
combination         combination156
Name: 155, dtype: object
For the categories: Musical_Instruments Video_Games
This category is recommended:  Kindle_Store
cat

In [149]:
def getCatById(id):
    return df_grouped.category.loc[df_grouped.category_id == id].iloc[0]
def getUserById(id):
    return df_grouped.reviewer.loc[df_grouped.reviewer_id == id].iloc[0]
def getIdByCat(cat):
    return df_grouped.category_id.loc[df_grouped.category == cat].iloc[0]
def getIdByUser(reviewer):
    return df_grouped.reviewer_id.loc[df_grouped.reviewer == reviewer].iloc[0]

In [166]:
df_lookUp.head()

,category1,category2,combination,recommended_category
0,Arts_Crafts_and_Sewing,Automotive,combination1,Luxury_Beauty
1,Arts_Crafts_and_Sewing,CDs_and_Vinyl,combination2,Automotive
2,Arts_Crafts_and_Sewing,Cell_Phones_and_Accessories,combination3,Kindle_Store
3,Arts_Crafts_and_Sewing,Clothing_Shoes_and_Jewelry,combination4,Automotive
4,Arts_Crafts_and_Sewing,Electronics,combination5,Luxury_Beauty


NameError: name 'df_fav_categories' is not defined

In [183]:
df_lookUp.to_csv('lookUp_table.csv', index=False)